In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Pipeline complet de traitement de données :
1. Chargement des fichiers bruts (annuel, calendrier)
2. Nettoyage, fusion et enrichissement -> exploratoire (explodata.csv)
3. Encodage des variables catégorielles -> finale (finaldata.csv)
"""

import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime
import holidays
from typing import Dict
from sklearn.preprocessing import LabelEncoder

def charger_dataframes(dep_folder: str) -> Dict[str, pd.DataFrame]:
    """
    Parcourt tous les fichiers CSV et Excel d'un dossier et renvoie un dict {nom_fichier: df}
    """
    d = {}
    patterns = [
        os.path.join(dep_folder, '*.csv'),
        os.path.join(dep_folder, '*.xlsx'),
        os.path.join(dep_folder, '*.xls'),
    ]
    for pattern in patterns:
        for filepath in glob.glob(pattern):
            name = os.path.splitext(os.path.basename(filepath))[0]
            ext = filepath.lower().split('.')[-1]
            if ext == 'csv':
                df = pd.read_csv(filepath, sep=None, engine='python')
            else:
                df = pd.read_excel(filepath)
            d[name] = df
            print(f"  • Chargé {filepath} -> {name} ({df.shape[0]} lignes)")
    return d

def formater_donnees(
    data_annuel: Dict[str, pd.DataFrame],
    calendrier: Dict[str, pd.DataFrame]
) -> pd.DataFrame:
    """
    Nettoie, fusionne et enrichit les données :
     - conserve périmètre France
     - parse Date, supprime les ND
     - fusion sur Date
     - ajoute Jour (nom), Saison, JourFerie
    """
    # 1) annuel
    df_ann = pd.concat(data_annuel.values(), ignore_index=True)
    df_ann.replace("ND", np.nan, inplace=True)
    df_ann = df_ann[df_ann["Périmètre"] == "France"]
    df_ann["Date"] = pd.to_datetime(df_ann["Date"], errors="coerce")
    df_ann.dropna(subset=["Date"], inplace=True)

    # 2) calendrier
    df_cal = pd.concat(calendrier.values(), ignore_index=True)
    df_cal["Date"] = pd.to_datetime(df_cal["Date"], errors="coerce")
    df_cal.dropna(subset=["Date"], inplace=True)

    # 3) fusion
    df = pd.merge(df_ann, df_cal, on="Date", how="inner")

    # 4) enrichissement temporel
    df["Jour"] = df["Date"].dt.day_name(locale='fr_FR')
    mois = df["Date"].dt.month
    conditions = [
        mois.isin([12, 1, 2]),
        mois.isin([3, 4, 5]),
        mois.isin([6, 7, 8]),
        mois.isin([9, 10, 11]),
    ]
    saisons = ["Hiver", "Printemps", "Été", "Automne"]
    df["Saison"] = np.select(conditions, saisons, default="Inconnu")

    # 5) jours fériés France
    years = df["Date"].dt.year.unique().tolist()
    fr_hol = holidays.FR(years=years)
    df["JourFerie"] = df["Date"].isin(fr_hol).astype(int)

    return df

def encoder_donnees(df: pd.DataFrame) -> pd.DataFrame:
    """
    Encode les colonnes catégorielles en entiers.
    """
    df_enc = df.copy()
    # Colonnes de base à encoder
    for col in ["Jour", "Saison", "Région", "Type", "Categorie"]:
        if col in df_enc.columns:
            le = LabelEncoder()
            df_enc[col] = le.fit_transform(df_enc[col].astype(str))
    return df_enc

def main():
    # 1) répertoires d'entrée et fichiers à produire
    folder_annuel     = './data/raw/annuel/'
    folder_calendrier = './data/raw/calendrier/'
    out_explora       = 'explodata.csv'
    out_final         = 'finaldata.csv'

    print("🔄 Chargement des données brutes…")
    data_annuel = charger_dataframes(folder_annuel)
    data_cal    = charger_dataframes(folder_calendrier)

    # Vérification pour éviter concat vides
    if not data_annuel:
        raise FileNotFoundError(f"Aucun fichier trouvé dans '{folder_annuel}'")
    if not data_cal:
        raise FileNotFoundError(f"Aucun fichier trouvé dans '{folder_calendrier}'")

    print("\n🧹 Nettoyage, fusion et enrichissement…")
    df_explora = formater_donnees(data_annuel, data_cal)
    df_explora.to_csv(out_explora, index=False, encoding='utf-8')
    print(f"✔ Exploratoire enregistré : {out_explora} ({df_explora.shape[0]} lignes, {df_explora.shape[1]} colonnes)")

    print("\n🔢 Encodage des variables catégorielles…")
    df_final = encoder_donnees(df_explora)
    df_final.to_csv(out_final, index=False, encoding='utf-8')
    print(f"✔ Final enregistré       : {out_final} ({df_final.shape[0]} lignes, {df_final.shape[1]} colonnes)")

if __name__ == "__main__":
    main()


🔄 Chargement des données brutes…


FileNotFoundError: Aucun fichier trouvé dans './data/raw/annuel/'